In [ ]:
import time
import os
import pickle
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
from collections import Counter
from bs4 import BeautifulSoup
import re
from datetime import datetime
from urllib.parse import urlparse, parse_qs, urlencode
# Initialize undetected Chrome
options = uc.ChromeOptions()
options.headless = False  # Change to True if you want headless mode
driver = uc.Chrome(options=options)

# Open the login page
driver.get("https://www.equipnet.com/login/")
time.sleep(5)  # Allow time for page load

# Attempt to detect an active session
already_logged_in = False
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[@href='/myequipnet/account/']"))
    )
    print("Already logged in! ✅")
    already_logged_in = True
except:
    print("No active session detected.")

# If not already logged in, load cookies (if available)
if not already_logged_in and os.path.exists("cookies.pkl"):
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)
    print("Cookies loaded! Refreshing to check login status...")
    driver.refresh()
    time.sleep(5)

# Check if login is now successful
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[@href='/myequipnet/account/']"))
    )
    print("Login successful! ✅")
    already_logged_in = True
except:
    print("Proceeding with manual login...")

# If still not logged in, enter credentials manually
if not already_logged_in:
    driver.find_element(By.ID, "Username").send_keys("goldenstaterecyclers")
    driver.find_element(By.ID, "Password").send_keys("Speed2cruisecontrol!")
    driver.find_element(By.ID, "login-submit").click()

    # Wait for login process
    time.sleep(10)

    # Verify login
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@href='/myequipnet/account/']"))
        )
        print("Login successful! ✅")

        # Save cookies for future use
        with open("cookies.pkl", "wb") as file:
            pickle.dump(driver.get_cookies(), file)
        print("Cookies saved! 🥳")

    except:
        print("Login failed! ❌ Exiting script.")
        driver.quit()
        exit()
    # Open the watchlist page
    watchurl = "https://www.equipnet.com/myequipnet/myoffers/auction-watchlist/"
    driver.get(watchurl)
    time.sleep(5)  # Allow page to load

    # Extract watchlist URLs
    watchlist_links = driver.find_elements(By.XPATH, '//a[contains(text(), "View Details")]')

    # Store and print extracted URLs
    watchlist_urls = [link.get_attribute("href") for link in watchlist_links]

    if not watchlist_urls:
        print("No watchlist items found! Exiting...")
        # driver.quit()
        # exit()

    print("Extracted Watchlist URLs:")
    for url in watchlist_urls:
        print(url)

    # Extract product links from each watchlist page
    product_links = []
    try:
        for url in watchlist_urls:
            driver.get(url)
            time.sleep(5)  # Wait for the page to load

            # Find all divs with class "row" (each row represents a product)
            row_divs = driver.find_elements(By.CLASS_NAME, "col-md-12.form-inline")

            for row_div in row_divs:
                # Find all <a> tags inside this row
                a_tags = row_div.find_elements(By.TAG_NAME, "a")

                # Extract href attributes
                for a in a_tags:
                    link = a.get_attribute("href")
                    if link and link not in product_links:  # Avoid duplicates
                        product_links.append(link)
        print("Extracted Product Links:")
        for link in product_links:
            print(link)
    except Exception as e:
        print("Error extracting product links:")
# Print the extracted product links

def scrape_product_data(url):
    driver.get(url)
    time.sleep(10)  # Wait for the page to load

    product_data = {
        "Lot Number": "Not found",
        "title": "Not found",
        "model": "Model not found",
        "Location": "Location not found",
        "Start Date": "Start Date not found",
        "End Date": "Not found",
        "Link": url
    }

    try:
        lot_number = driver.find_element(By.CSS_SELECTOR, "h1.list-view-heading span.orange").text
        product_data["Lot Number"] = lot_number.replace("Lot ", "").split()[0].strip()
    except:
        pass

    try:
        title = driver.find_element(By.CSS_SELECTOR, "h1.list-view-heading").text
        product_data["Title"] = title.replace(lot_number, "").strip()
    except:
        pass

    try:
        model = driver.find_element(By.XPATH, "//div[contains(text(), 'Model:')]/following-sibling::div[1]").text
        product_data["Model"] = model.strip()
    except:
        pass

    try:
        start_date = driver.find_element(By.XPATH, "//h6[contains(text(), 'Event Start Date:')]/following-sibling::p/span").text
        product_data["Start Date"] = start_date.strip()
    except:
        pass

    try:
        location = driver.find_element(By.XPATH, "//h6[contains(text(), 'Location:')]/following-sibling::p").text
        product_data["Location"] = location.strip()
    except:
        pass
    
    return product_data

# Scrape data from each product URL
all_product_data = []
for url in product_links:
    data = scrape_product_data(url)
    if data:  # Only append if data was successfully scraped
        all_product_data.append(data)

# Create a DataFrame
df1 = pd.DataFrame(all_product_data, columns=["Lot Number", "title", "model", "Location", "Start Date", "End Date", "Link"])

# Save data to CSV file
csv_filename = "product_data.csv"
df1.to_csv(csv_filename, index=False, encoding="utf-8")
print(df1)
print(f"Data saved to {csv_filename} successfully! ✅")

# Close the WebDriver
driver.quit()

In [ ]:
########################################################
########################################################
#################  EBAY CODE BELOW   ###################
########################################################
########################################################
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-notifications")  # Disable notifications

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)


def switch_tab_to_active(driver):
    # Get the current URL
    current_url = driver.current_url

    # Parse the URL
    parsed_url = urlparse(current_url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Check and modify the 'tabName' parameter
    if 'tabName' in query_params:
        current_tab = query_params['tabName'][0]
        if current_tab == 'SOLD':
            query_params['tabName'][0] = 'ACTIVE'
        elif current_tab == 'ACTIVE':
            query_params['tabName'][0] = 'SOLD'
        else:
            print(f"Tab name is not 'SOLD' or 'ACTIVE', it's: {current_tab}")
    else:
        # If 'tabName' is not present, set it to 'ACTIVE' by default
        query_params['tabName'] = ['ACTIVE']

    # Reconstruct the URL with modified parameters
    new_query = urlencode(query_params, doseq=True)
    new_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{new_query}"

    # Navigate to the new URL
    driver.get(new_url)
    print(f"Navigated to: {driver.current_url}")

def fetch_price_and_process(driver, average_listed_prices, average_sold_prices):
    switch_tab_to_active(driver)

    # Parse the URL
    current_url = driver.current_url
    parsed_url = urlparse(current_url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Check and modify the 'tabName' parameter
    if 'tabName' in query_params:
        current_tab = query_params['tabName'][0]
        if current_tab == 'SOLD':
            query_params['tabName'][0] = 'ACTIVE'  # Switch tab to 'ACTIVE' if 'SOLD' is present
            is_sold = True
        elif current_tab == 'ACTIVE':
            query_params['tabName'][0] = 'SOLD'  # Switch tab to 'SOLD' if 'ACTIVE' is present
            is_sold = False
        else:
            print(f"Tab name is not 'SOLD' or 'ACTIVE', it's: {current_tab}")
    else:
        # If 'tabName' is not present, set it to 'ACTIVE' by default
        query_params['tabName'] = ['ACTIVE']
        is_sold = False  # Default to 'ACTIVE'

    # Process page and extract prices
    time.sleep(5)
    current_url = driver.current_url
    try:
        try:
            # Wait for the tourtip to appear
            tourtip = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".tourtip__mask"))
            )
            
            # Find the close button within the tourtip
            close_button = tourtip.find_element(By.CSS_SELECTOR, ".tourtip__close")
            
            # Click on the close button
            close_button.click()
            
            print("Tourtip closed successfully.")
        except:
            pass
        time.sleep(3)
        html_content = driver.page_source
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the .aggregates div
        aggregates = soup.find('div', class_='aggregates')
        
        if aggregates:
            # Find the first .metric-value element
            metric_value = aggregates.find('div', class_='metric-value')
            
            if metric_value:
                # Extract the text content
                avg_price = metric_value.text.strip()
                if is_sold:
                    average_sold_prices.append(avg_price)
                    print(f"Avg Sold Price: ${avg_price}")
                else:
                    average_listed_prices.append(avg_price)
                    print(f"Avg Listed Price: ${avg_price}")
            else:
                print("Metric value not found.")
                if is_sold:
                    average_sold_prices.append('0')
                else:
                    average_listed_prices.append('0')
        else:
            print("Aggregates div not found.")
            if is_sold:
                average_sold_prices.append('0')
            else:
                average_listed_prices.append('0')

    except Exception as e:
        print(f'An error occurred while trying to get the average listing price: {str(e)}')




# Navigate to eBay research page
url = "https://www.ebay.com/sh/research?marketplace=EBAY-US&tabName=SOLD"
driver.get(url)
print('waiting for 15 seconds')
time.sleep(60)
print('waiting done for 15 seconds')

# Wait for email field to be present
email_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "userid"))
)

# Fill in email address
email_field.send_keys("zainiawan5@gmail.com")

# Locate Continue button
continue_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "signin-continue-btn"))
)

# Click Continue button
continue_button.click()
time.sleep(30)
# Wait for password field to be present
password_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "pass"))
)

# Fill in password
password_field.send_keys("254ed228")

# Locate Sign in button
sign_in_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "sgnBt"))
)

# Click Sign in button
sign_in_button.click()

time.sleep(30)
try:
    try:
        # Wait for Skip for now button to be clickable
        skip_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "passkeys-cancel-btn"))
        )

        # Click Skip for now button
        skip_button.click()
        time.sleep(30)
    except:
        pass
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    # for index, row in df.iterrows():    
    #     try:
    #         search_input = WebDriverWait(driver, 10).until(
    #             EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
    #         )
    #         search_input.clear()
    #         search_input.send_keys(f"{row['title']}, {row['model']}")
    #         time.sleep(3)
    #         research_button = WebDriverWait(driver, 10).until(
    #             EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-input-panel__research-button"))
    #         )
    #         driver.execute_script("arguments[0].scrollIntoView(true);", research_button)
    #         research_button.click()            
            
    #     except:
    #         print('erro in the sending text try catch')
    average_listed_prices=[]
    average_sold_prices=[]


    for index, row in df1.iterrows():    
        try:
            # search_input = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
            # )
            # # Clear the existing text before sending new text
            # search_input.clear()
            # search_input.send_keys(f"{row['title']}, {row['model']}")
            
            search_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
            )
            # Clear the existing text using JavaScript
            driver.execute_script("arguments[0].value = ''; arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", search_input)
            # Check if 'model' exists, is not empty, and does not contain "Model not found"
            if 'model' in row and row['model'].strip() and row['model'].strip().lower() != "model not found":
                # Search by both model and title
                search_input.send_keys(f"{row['model']} {row['title']}")
            else:
                # Search only by title
                search_input.send_keys(f"{row['title']}")      
            
            
            time.sleep(3)
            research_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-input-panel__research-button"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", research_button)
            research_button.click()            
        except:
            print('error in the sending text try catch')

















        # Function to fetch and process the price


        for _ in range(2):
            fetch_price_and_process(driver, average_listed_prices, average_sold_prices)

        # You can check the result
        print("Avg Listed Prices:", average_listed_prices)
        print("Avg Sold Prices:", average_sold_prices)





















        switch_tab_to_active(driver)
        time.sleep(3)
except Exception as e:
    print(f"An error occurred: {e}")
# Close browser after processing
driver.quit()




df2_columns = pd.DataFrame({
    'average_listed_price': average_listed_prices,
    'average_sold_prices': average_sold_prices
})

# Concatenate only the last two columns to df
df = pd.concat([df1, df2_columns], axis=1)
filename = 'product_data.csv'
df.to_csv(filename, index=False)
print(f"\nData saved to {filename}")